In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import dateutil.parser as parser
import numpy as np
from matplotlib import pyplot as plt

#### Helper functions:

In [2]:
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield eval(l)

### Evaluation metrics:
def MSE(Y1, Y2):
    return np.mean((Y1-Y2)**2)

def binary_error_rate(Ypred, Ytest):
    # Check binary error rate, see report section 2
    TP = sum( np.logical_and(Ypred>=4.0, Ytest>=4.0) )
    FP = sum( np.logical_and(Ypred>=4.0, Ytest<4.0) )
    TN = sum( np.logical_and(Ypred<4.0, Ytest<4.0) )
    FN = sum( np.logical_and(Ypred<4.0, Ytest>=4.0) )

    assert TP+FP+TN+FN == len(Ytest)

    Accuracy = (TP + TN) / len(Ytest)
    BER = 1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))
    print(f"TP:{TP}, FP:{FP}, TN:{TN}, FN:{FN}")
    print(f"Accuracy:{Accuracy}, BER:{BER}")
    
    
def round_predictions(predictions):
    '''
    Round predictions to the nearest integer
    '''
    rounded_predictions = np.zeros_like(predictions)
    for i, pred in enumerate(predictions):
        rounded_predictions[i] = int(round(pred))
    return rounded_predictions

def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

#### Get data

In [3]:
# review data
data = []
for d in parse("./lthing_data/reviews.json"):
    # filter review without rating
    if ('stars' not in d) or d['stars'] == None: continue
    # or without valid date
    if ('unixtime' not in d) or (d['unixtime'] == None) or (d['unixtime'] == -86400): continue

    # for this moment, we don't need actual text
    d['length'] = len(d['comment']) # store the length
    del d['comment']
    
    # train and test
    data.append(d)

print(f"Total number of reviews with a rating: {len(data)}")
data[0]

Total number of reviews with a rating: 1383597


{'work': '3206242',
 'flags': [],
 'unixtime': 1194393600,
 'stars': 5.0,
 'nhelpful': 0,
 'time': 'Nov 7, 2007',
 'user': 'van_stef',
 'length': 83}

In [4]:
# 8:2 train-test split
cut = int(len(data) * 0.8)
train_data, test_data = data[:cut], data[cut:]
del data # save memory

In [5]:
# social network data
with open('lthing_data/edges.txt') as f:
    edges = f.readlines()
    
friendsPerUser = defaultdict(set)
for pair in edges:
    friends = pair.strip().split()
    friendsPerUser[friends[0]].add(friends[1])
    friendsPerUser[friends[1]].add(friends[0])

### Populate useful data structures.

In [6]:
# for Jaccard similarity
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

# incorporate the timestamp of the review
usersTimePerItem = defaultdict(list) 
itemsTimePerUser = defaultdict(list)

# very likely to have same ratings, so use list
ratingsPerItem = defaultdict(list)
ratingsPerUser = defaultdict(list)

# Each User/Item has a list of length 3: nhelpful, #abuse, #not_a_review
recordPerUser = defaultdict(lambda:[0,0,0])
recordPerItem = defaultdict(lambda:[0,0,0])

for d in train_data:
    
    u, i, r, dt, dl = d['user'], d['work'], d['stars'], parser.parse(d['time']), d['length']
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)
    usersTimePerItem[i].append( (dt, u, dl) )
    itemsTimePerUser[u].append( (dt, i, dl) )
    ratingsPerItem[i].append(r)
    ratingsPerUser[u].append(r)
    
    if d['nhelpful']:
        recordPerUser[u][0] += d['nhelpful']
        recordPerItem[i][0] += d['nhelpful']
    if d['flags']:
        if 'abuse' in d['flags']:
            recordPerUser[u][1] += 1
            recordPerItem[i][1] += 1
        if 'not_a_review' in d['flags']:
            recordPerUser[u][2] += 1
            recordPerItem[i][2] += 1

# calculate 2 global averages for cold-start user or book: average of each user's/book's average ratings
avgBookRating = sum( sum(ratingsPerItem[i])/len(ratingsPerItem[i]) for i in ratingsPerItem)/len(ratingsPerItem)
avgUserRating = sum( sum(ratingsPerUser[u])/len(ratingsPerUser[u]) for u in ratingsPerUser)/len(ratingsPerUser)

print(avgBookRating, avgUserRating)

3.7653503816851766 3.9605122606260252


In [7]:
print(f"A total of {len(recordPerUser)} users out of {len(itemsPerUser)} have helpful votes, abuse review, or not-a-review.")
print(f"A total of {len(recordPerItem)} items out of {len(usersPerItem)} have helpful votes, abuse review, or not-a-review.")

A total of 26049 users out of 65546 have helpful votes, abuse review, or not-a-review.
A total of 54560 items out of 333400 have helpful votes, abuse review, or not-a-review.


In [8]:
# Sort interaction data structures by time for later use
for i in usersTimePerItem:
    usersTimePerItem[i].sort()
    
for u in itemsTimePerUser:
    itemsTimePerUser[u].sort()

# baseline model

### A relevant simple baseline that predicts rating based on the average rating given by a user, and the average rating received by a book.

In [9]:
def feature_baseline(d):
    global avgBookRating, avgUserRating
    result = [1] # bias term
    u, i = d['user'], d['work']
    if u in ratingsPerUser:
        result.append( sum(ratingsPerUser[u]) / len(ratingsPerUser[u]) )
    else:
        result.append(avgUserRating)
    if i in ratingsPerItem:
        result.append( sum(ratingsPerItem[i]) / len(ratingsPerItem[i]) )
    else:
        result.append(avgBookRating)
    return result

In [10]:
Xtrain_baseline = np.array( [feature_baseline(d) for d in train_data] )
Xtest_baseline = np.array( [feature_baseline(d) for d in test_data] )

Ytrain = np.array( [d['stars'] for d in train_data] )
Ytest = np.array( [d['stars'] for d in test_data] )

In [11]:
model_baseline = linear_model.Ridge(1, fit_intercept=False)
model_baseline.fit(Xtrain_baseline, Ytrain)

model_baseline.coef_

array([-1.87617111,  0.6602512 ,  0.83293227])

In [12]:
# Evaluate on train set
Ypred_baseline_train = model_baseline.predict(Xtrain_baseline)
print("Ypred baseline train:")
print("MSE = ", MSE(Ypred_baseline_train, Ytrain))
binary_error_rate(Ypred_baseline_train, Ytrain)

Ypred baseline train:
MSE =  0.49414739601287017
TP:394096, FP:36270, TN:381454, FN:295057
Accuracy:0.7006650242077485, BER:0.2574860347308826


In [13]:
# round to nearest integers
rounded_Ypred_baseline_train = round_predictions(Ypred_baseline_train)
print("Rounded Ypred baseline train:")
print("MSE = ", MSE(rounded_Ypred_baseline_train, Ytrain))
binary_error_rate(rounded_Ypred_baseline_train, Ytrain)

Rounded Ypred baseline train:
MSE =  0.5692389488624301
TP:623479, FP:167585, TN:250139, FN:65674
Accuracy:0.7892638477446003, BER:0.24824132040848057


In [14]:
# Evaluate on test set
Ypred_baseline_test = model_baseline.predict(Xtest_baseline)
print("Ypred baseline test:")
print("MSE = ", MSE(Ypred_baseline_test, Ytest))
binary_error_rate(Ypred_baseline_test, Ytest)

Ypred baseline test:
MSE =  0.8487029094124011
TP:79748, FP:17570, TN:86385, FN:93017
Accuracy:0.6003649898814686, BER:0.3537086573763284


In [15]:
# round Ypred to nearest integers
rounded_Ypred_baseline_test = round_predictions(Ypred_baseline_test)
print("Rounded Ypred baseline test:")
print("MSE = ", MSE(rounded_Ypred_baseline_test, Ytest))
binary_error_rate(rounded_Ypred_baseline_test, Ytest)

Rounded Ypred baseline test:
MSE =  0.9395164787510841
TP:147292, FP:62277, TN:41678, FN:25473
Accuracy:0.6828924544666088, BER:0.3732597909927997


In [23]:
# Compare it to a even more trivial baseline: always predict median 4.0
trivial_baseline = np.array([4.0]*len(Ytest))
print("Trivial baseline:")
print("MSE = ", MSE(trivial_baseline, Ytest))
binary_error_rate(trivial_baseline, Ytest)

Trivial baseline:
MSE =  1.0329249783174328
TP:172765, FP:103955, TN:0, FN:0
Accuracy:0.6243314541775079, BER:0.5


In [24]:
trivial_baseline_train = np.array([4.0]*len(Ytrain))
print("Trivial baseline:")
print("MSE = ", MSE(trivial_baseline_train, Ytrain))
binary_error_rate(trivial_baseline_train, Ytrain)

Trivial baseline:
MSE =  1.038931606673551
TP:689153, FP:417724, TN:0, FN:0
Accuracy:0.6226102809977984, BER:0.5


### Basic feature engineering design

In [17]:
""" Integrate features relavent to a user """
def featureU(u, t):
    global avgUserRating
    if u not in ratingsPerUser:
        # cold-start user, see below
        return [1, avgUserRating, 0,0,0,0]
    
    f = [1] # add a bias term
    # average rating this user gives; average of all users' average ratings (see baseline)
    f.append( sum(ratingsPerUser[u]) / len(ratingsPerUser[u]) )
    
    # This user's number of 'not_a_review' and 'abuse' comments respectively; 0, 0
    # Let the model decide their effect on rating prediction
    f += recordPerUser[u]
    # nhelpful received; 0
    
    # ??? time (maybe in month?) since his last reading; 
    
    # number of books he has read till this time t; 0
    # General opinion (rating habit may change as one read more books)
    c = 0
    dt = parser.parse(t)
    while c<len(itemsTimePerUser[u]) and dt > itemsTimePerUser[u][c][0]: 
        c += 1
    f.append(c)
    
    assert(len(f) == 6)
    return f

In [18]:
# test featureU
[featureU(d['user'], d['time']) for d in test_data[:5]]

[[1, 2.5427350427350426, 31, 0, 5, 10],
 [1, 3.274193548387097, 9, 0, 0, 5],
 [1, 3.5917874396135265, 173, 0, 0, 116],
 [1, 3.9342105263157894, 13, 0, 1, 23],
 [1, 4.31875, 1, 0, 1, 74]]

In [19]:
""" Integrate features relavent to a book """
def featureI(i, t):
    global avgBookRating
    if i not in ratingsPerItem:
        # cold-start book
        return [avgBookRating, 0,0,0,0]
    
    f = []
    # average rating this book receives; average of all books' average ratings (see baseline)
    f.append( sum(ratingsPerItem[i]) / len(ratingsPerItem[i]) )
    
    # This item's nhelpful vote
    f.append(recordPerItem[i][0])
    # nhelpful received; 0
    
    # length of time interval (in month) this book was read by people (t_last_read - t_first_read); 0
    diff_times = usersTimePerItem[i][-1][0] - usersTimePerItem[i][0][0]
    f.append( diff_times.days / 30 )
    
    # average length of the comment it received; 0
    all_lengths = [user[2] for user in usersTimePerItem[i]]
    f.append( sum(all_lengths) / len(all_lengths) )
    
    # number of users that have read this book till this time t; 0
    # General opinion (rating may change as a book is read more times)
    c = 0
    dt = parser.parse(t)
    while c<len(usersTimePerItem[i]) and dt > usersTimePerItem[i][c][0]: 
        c += 1
    f.append(c)
    
    assert(len(f) == 5)
    return f

In [20]:
# test featureI
[featureI(d['work'], d['time']) for d in test_data[:5]]

[[4.0, 0, 0.0, 294.0, 1],
 [4.625, 0, 64.33333333333333, 766.25, 0],
 [4.484795321637427, 625, 62.56666666666667, 890.1280701754386, 616],
 [3.9272727272727272, 3, 26.9, 1122.3636363636363, 31],
 [3.8990384615384617, 25, 24.466666666666665, 1121.6153846153845, 95]]

In [21]:
""" Integrate features relavent to user-item interactions """
def featureInter(u, i):
    f = []
    # Book/item similarity (max similarity between i and the books that u has read)
    book_max_sim = 0
    if u in itemsPerUser:
        for book in itemsPerUser[u]:
            if book == i: continue
            sim = Jaccard(usersPerItem[i], usersPerItem[book])
            if sim > book_max_sim:
                book_max_sim = sim
    f.append(book_max_sim)
    
    # User similarity (max similarity between u and the users that have read i)
    user_max_sim = 0
    if i in usersPerItem:
        for user in usersPerItem[i]:
            if user == u: continue
            sim = Jaccard(itemsPerUser[u], itemsPerUser[user])
            if sim > user_max_sim:
                user_max_sim = sim
    f.append(user_max_sim)
    
    # Number of u’s friends that have read i
    num_friends = 0
    if u in friendsPerUser:
        for friend in friendsPerUser[u]:
            if friend in usersPerItem[i]:
                num_friends += 1
    f.append(num_friends)
    
    # Average rating u’s friends gives
    sum_friends_rating = 0
    num_friends_rating = 0
    if u in friendsPerUser:
        for friend in friendsPerUser[u]:
            if friend in ratingsPerUser:
                sum_friends_rating += sum(ratingsPerUser[friend])
                num_friends_rating += len(ratingsPerUser[friend])
    avg_friends_rating = 0
    if num_friends_rating == 0:
        f.append(0)
    else:
        f.append(sum_friends_rating / num_friends_rating)
    
    return f

In [22]:
# test featureInter
[featureInter(d['user'], d['work']) for d in test_data[:5]]

[[0.0024875621890547263, 0.0043859649122807015, 0, 3.8947784810126582],
 [0.01098901098901099, 0.0032258064516129032, 0, 0],
 [0.08033240997229917, 0.04234527687296417, 0, 3.839939024390244],
 [0.0375, 0.02654867256637168, 0, 3.9117161716171616],
 [0.04591836734693878, 0.0379746835443038, 0, 3.8333333333333335]]

### Test new features

In [87]:
Xtrain_featureUI = np.array( [featureU(d['user'], d['time']) + featureI(d['work'], d['time']) + featureInter(d['user'], d['work']) for d in train_data] )
Xtest_featureUI = np.array( [featureU(d['user'], d['time']) + featureI(d['work'], d['time']) + featureInter(d['user'], d['work']) for d in test_data] )

In [88]:
model_featureUI = linear_model.Ridge(1, fit_intercept=False)
model_featureUI.fit(Xtrain_featureUI, Ytrain)

model_featureUI.coef_

array([-1.87930251e+00,  6.60209984e-01, -1.24935129e-05,  2.09952691e-04,
       -6.47237432e-04, -2.13446597e-04,  8.33901735e-01,  7.61556072e-06,
        1.40845145e-05, -1.31423936e-05, -1.67225930e-03,  2.85758755e-02,
        4.90432332e-03])

In [89]:
# Evaluate on train set
Ypred_featureUI_train = model_featureUI.predict(Xtrain_featureUI)
print("Ypred featureUI train:")
print("MSE = ", MSE(Ypred_featureUI_train, Ytrain))
binary_error_rate(Ypred_featureUI_train, Ytrain)

Ypred featureUI train:
MSE =  0.49344140592319824
TP:395455, FP:36656, TN:381068, FN:293698
Accuracy:0.7015440740028025, BER:0.25696206933898225


In [90]:
# round to nearest integers
rounded_Ypred_featureUI_train = round_predictions(Ypred_featureUI_train)
print("Rounded Ypred featureUI train:")
print("MSE = ", MSE(rounded_Ypred_featureUI_train, Ytrain))
binary_error_rate(rounded_Ypred_featureUI_train, Ytrain)

Rounded Ypred featureUI train:
MSE =  0.5682496790519633
TP:623279, FP:167287, TN:250437, FN:65874
Accuracy:0.7893523851340303, BER:0.24802973121602023


In [91]:
# Evaluate on test set
Ypred_featureUI_test = model_featureUI.predict(Xtest_featureUI)
print("Ypred featureUI test:")
print("MSE = ", MSE(Ypred_featureUI_test, Ytest))
binary_error_rate(Ypred_featureUI_test, Ytest)

Ypred featureUI test:
MSE =  0.8487489794325093
TP:80113, FP:17676, TN:86279, FN:92652
Accuracy:0.6013009540329575, BER:0.35316214514437483


In [92]:
# round to nearest integers
rounded_Ypred_featureUI_test = round_predictions(Ypred_featureUI_test)
print("Rounded Ypred featureUI test:")
print("MSE = ", MSE(rounded_Ypred_featureUI_test, Ytest))
binary_error_rate(rounded_Ypred_featureUI_test, Ytest)

Rounded Ypred featureUI test:
MSE =  0.9396574154379879
TP:147235, FP:62392, TN:41563, FN:25530
Accuracy:0.6822708875397514, BER:0.3739778789090582
